# Tarea 1

**Integrante:** Danny Fuentes

# Drive

In [1]:
use_drive = True
path = ""
from google.colab import drive
if(use_drive):
  path = "/content/drive/MyDrive/Universidad/Procesamiento Masivo/tarea1"
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Imports

In [2]:
!pip install emoji
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import string
from datetime import datetime
import emoji
from itertools import combinations
from unidecode import unidecode

from random import shuffle
from random import seed
seed(42)

import nltk
from nltk import ngrams
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data Handling

In [4]:
data = pd.read_csv(f'{path}/tweets_2022_abril_junio.csv')

In [5]:
data[["user", "text"]] = data[["screen_name", "text"]]

In [6]:
data = data[["user", "text"]]

In [7]:
data.shape

(4594980, 2)

In [8]:
data.head(3)

,user,text
0,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...


In [9]:
data.value_counts()

user           text                                                                                                                                         
ygorayeb       RT @ygorayeb: Listo....si Teillier, el que realmente manda, no quiere retiro...100% confirmado que se los van a robar...porque comunista pr…     1617
Miltonterasss  #RechazoDeSalida                                                                                                                                  340
x1educalidad   #Apruebo4deSeptiembre2022 \n#AprueboPlebiscitoDeSalida \n#AprueboDeSalida \n#Apruebo \n#SiPoApruebo                                               223
Miltonterasss  #RechazoDeSalida #RechazoElMamarracho                                                                                                             200
ClauDePizan    #RechazoDeSalida                                                                                                                                  185
                  

In [10]:
data = data.drop_duplicates(subset='text', keep='first')

In [11]:
data.shape

(1549216, 2)

In [12]:
def normalize(text):
  return unidecode(text)

In [13]:
def cleanPunctuation(text):
  tabla = str.maketrans("", "", string.punctuation+'́')
  return text.translate(tabla)

Limpieza de tweets.
- Se eliminan los tags a otros usuarios
- Se elimina la palabra RT
- Se elimina la puntuacion
- Se normaliza el texto, eliminar tildes y emojis
- Se eliminan todos los tweets de usuarios que tengan menos de 20 tweets

In [14]:
data["text"] = data["text"].apply(lambda line: " ".join([word for word in line.split() if "@" not in word and "RT" not in word and "https://" not in word]))
data["text"] = data["text"].apply(lambda line: cleanPunctuation(line))

In [15]:
data["text"] = data["text"].apply(lambda line: normalize(line))
data["text"] = data["text"].apply(lambda line: line.lower())
data["text"] = data["text"].apply(lambda line: emoji.replace_emoji(line, replace=''))

In [16]:
# filtrar usuarios por cantidad de tweets
counts = data['user'].value_counts()

user_filtered = counts[counts >= 20].index.tolist()
data = data[data['user'].isin(user_filtered)]

In [17]:
len(set(data['user']))

16876

In [18]:
data.shape

(1099386, 2)

# Generating Vocab

### Generating Shingles

Debido a lo masivo de los datos, preferí un enfoque de generar shingles por palabras, en este caso, me quede con largo 1 para hacerlo mas simple.

In [19]:
k = 1
data["shingles"] = data["text"].apply(lambda line: [" ".join(x) for x in list(set(ngrams(line.split(), k)))])

In [20]:
data.head(3)

,user,text,shingles
2,Cesar_A_RR,aqui esta el aporte de con respecto a los dere...,"[que, negan..., de, respecto, los, nueva, esta..."
4,GQuelluen,ante la circulacion de noticias falsas les qui...,"[falsas, les, noticias, de, circulacion, mis, ..."
7,MariliaBloden,perdon pero viene faltando a la verdad desde l...,"[semana, verific..., perdon, de, tuvo, pero, v..."


In [21]:
def save_df_tocsv(df, prefix="signatures"):
  timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
  output_file = f"{path}/{prefix}_{timestamp}.csv"
  df.to_csv(output_file, index=False)

### Generating Vocab

Generacion de vocabulario, se toma un conjunto sin repeticiones de todas las palabras.

In [22]:
vocab = data['shingles'].explode()

In [23]:
vocab = list(set(vocab))

In [24]:
len(vocab)

217583

In [25]:
vocab_map = {}
for i in range(len(vocab)):
  vocab_map[vocab[i]] = i

In [26]:
len_vocab = len(vocab_map.keys())
len_vocab

217583

In [27]:
# vocab = None

In [28]:
# data = data.drop("text", axis=1)

# Signatures

Generacion de funciones de hashing universal. Basado en la explicaciones y codigo de un articulo de pinecone.
https://www.pinecone.io/learn/locality-sensitive-hashing/

In [29]:
def create_hash_func(size):
    hash_ex = list(range(1, size+1))
    shuffle(hash_ex)
    shuffle_dict = {}
    for i in range(len(hash_ex)):
      shuffle_dict[i] = hash_ex[i]
    return shuffle_dict

def build_minhash_func(vocab_size, nbits):
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

Se define que el largo de las firmas será de 64, estuve probando con valores cercanos a 100 pero el tiempo de computo vs el resultado no era tan significativo.

In [30]:
n = 64

In [31]:
seed(42)
minhash_func = build_minhash_func(len_vocab, n)

In [32]:
def get_signature(row, minhash_func):
  signature = ""

  for minhash in minhash_func:
    min = np.inf
    for shingle in row["shingles"]:
      vocab_index = vocab_map[shingle]
      shingle_minhash = minhash[vocab_index]
      if shingle_minhash < min:
        min = shingle_minhash
    signature += str(min) + " "
  return signature

In [33]:
data.head(3)

,user,text,shingles
2,Cesar_A_RR,aqui esta el aporte de con respecto a los dere...,"[que, negan..., de, respecto, los, nueva, esta..."
4,GQuelluen,ante la circulacion de noticias falsas les qui...,"[falsas, les, noticias, de, circulacion, mis, ..."
7,MariliaBloden,perdon pero viene faltando a la verdad desde l...,"[semana, verific..., perdon, de, tuvo, pero, v..."


In [34]:
data.shape

(1099386, 3)

Generación de las firmas

In [35]:
# %%time
data["signature"] = data.apply(lambda row: get_signature(row, minhash_func), axis=1)

In [36]:
save_df_tocsv(data, "signatures")

In [37]:
data.head(3)

,user,text,shingles,signature
2,Cesar_A_RR,aqui esta el aporte de con respecto a los dere...,"[que, negan..., de, respecto, los, nueva, esta...",15607 6120 10889 29762 213 4639 1878 11697 156...
4,GQuelluen,ante la circulacion de noticias falsas les qui...,"[falsas, les, noticias, de, circulacion, mis, ...",9146 4058 6968 6813 26780 16581 14921 11697 60...
7,MariliaBloden,perdon pero viene faltando a la verdad desde l...,"[semana, verific..., perdon, de, tuvo, pero, v...",15607 2052 11942 12320 26780 7774 4691 1410 76...


In [38]:
# data = data[["user", "signature"]]

# LSH

In [39]:
# data = pd.read_csv(f"{path}/signatures_20230607050147.csv")

In [40]:
data.head(1)

,user,text,shingles,signature
2,Cesar_A_RR,aqui esta el aporte de con respecto a los dere...,"[que, negan..., de, respecto, los, nueva, esta...",15607 6120 10889 29762 213 4639 1878 11697 156...


Ajuste de parametros del método

In [41]:
b = 16
r = n/b
t = (1/b)**(1/r)
print(f"b: {b}\nr: {r}\nt: {t}")

b: 16
r: 4.0
t: 0.5


In [42]:
buckets = []
for _ in range(b):
  buckets.append({})

Función que realiza el hasing usando el concepto de colisiones en un diccionario

In [43]:
def add_hash(row, bucket_index, r):
  signature = row["signature"].split()
  lower = int(bucket_index*r)
  upper = int((bucket_index+1)*r)
  signature = signature[lower:upper]
  signature = " ".join(signature)
  if signature not in buckets[bucket_index]:
    buckets[bucket_index][signature] = set()

  buckets[bucket_index][signature].add(row.name)

In [44]:
for i in range(b):
  data.apply(lambda row: add_hash(row, i, r), axis=1)

Se eliminan todos los hashes que tengan menos de 5 tweets

In [45]:
clean_buckets = []
for bucket in buckets:
  bucket_copy = bucket.copy()
  for sig in bucket:
    if len(bucket[sig]) <= 5:
      bucket_copy.pop(sig)
  clean_buckets.append(bucket_copy)

In [46]:
len(clean_buckets[0])

10710

Se genera una tabla de hashing en que la llave es la firma del tweet y los valore son todos los usuarios que colisionaron en esa firma

In [47]:
user_buckets = []
for bucket in clean_buckets:
  bucket_user = {}
  for sig in bucket:
    if sig not in bucket_user:
      bucket_user[sig] = set()
    for row_idx in bucket[sig]:
      bucket_user[sig].add(data.loc[row_idx]["user"])
  user_buckets.append(bucket_user)

In [ ]:
user_buckets[0]

Se buscan los usuarios que mas se repiten dentro de cada bucket. En este caso, intersectando los conjuntos de usuarios para cada firma, hasta mantener una cantidad especifica. En este caso 5 usuarios.

In [49]:
top_users = []
for bucket in user_buckets:
  inter_global = set(data["user"])
  for sig in bucket:
    tmp = inter_global.intersection(bucket[sig])
    if len(tmp) < 5:
      continue
    else:
      inter_global = tmp
  top_users.append(inter_global)

In [ ]:
top_users

Entre cada hash, se buscan los ids de los tweets que pertenecen a los usuarios encontrados como más frecuentes.

In [51]:
top_sigs_buckets = []
for i in range(len(clean_buckets)):
  top = top_users[i]
  bucket = clean_buckets[i]
  sig_bucket = {}
  top_idxs = set(data[data['user'].isin(top)].index.tolist())
  for sig in bucket:
    sig_idxs = set(bucket[sig])
    inter = top_idxs.intersection(sig_idxs)
    users = data.loc[list(inter)]["user"]
    if len(users) > 1:
      sig_bucket[sig] = inter 
  top_sigs_buckets.append(sig_bucket)

In [ ]:
top_sigs_buckets[0]

In [53]:
data = data[["user", "text"]]

Se imprimen los tweets de los usuarios mas frecuentes por bucket.

In [55]:
bucket = 1
tmp_keys = list(top_sigs_buckets[bucket].keys())
for i in range(len(tmp_keys)):
  sign = tmp_keys[i]
  print(data.loc[list(top_sigs_buckets[bucket][sign])])
  print("--------------------------------------------------")

                    user                                text
3089315    GODPorSiempre                           excelente
2405928        edotrigol                           excelente
4037802        edotrigol                           excelente
1697035  gloria_patriota  excelente rechazoelplurimamarracho
2487629  Veronic30786311                           excelente
4197678        edotrigol                          excelente 
563920        Eskol69903                          excelente 
3904978      jescorpionz                           excelente
564947            yplaza                          excelente 
2622935     michel778899                           excelente
1399801        edotrigol                           excelente
2320411  MarioPu22321263                           excelente
2608316        edotrigol                           excelente
857438   gloria_patriota                          excelente 
4391007        edotrigol                          excelente 
------------------------